# Introduction to Google's ADK - Part I

In [9]:
# Import necessary libraries
import os
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts
from typing import Optional, Dict, Any

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

Libraries imported.


In [10]:
MODEL_GPT = "openai/gpt-4o"

llm = LiteLlm(model=MODEL_GPT)
llm_small = LiteLlm(model="openai/gpt-4o-mini")
# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, 
                                messages=[{"role": "user", 
                                           "content": "Are you ready?"}], 
                                tools=[]))


print("\n OpenAI is ready")

ModelResponse(id='chatcmpl-CAX1RmewpwkaberClG64zOk6tulvx', created=1756626253, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_cbf1785567', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Yes, I'm ready! How can I assist you today?", role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={})], usage=Usage(completion_tokens=13, prompt_tokens=27, total_tokens=40, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

 OpenAI is ready


## 3.2. Explore `neo4j_for_adk`

In [11]:
from neo4j_for_adk import graphdb


In [12]:
neo4j_is_ready = graphdb.send_query("RETURN 'Neo4j is Ready!' as message1")

print(neo4j_is_ready)

{'status': 'success', 'query_result': [{'message1': 'Neo4j is Ready!'}]}


In [9]:
def wish(name: Optional[str] = "User") -> str:
    """Function to wish the user."""
    return f"Hello, {name}! How can I assist you today?"

In [10]:
wish("Rajesh")

'Hello, Rajesh! How can I assist you today?'

In [13]:
# Define a basic tool -- send a parameterized cypher query
def say_hello(person_name: str) -> dict:
    """Formats a welcome message to a named person. 

    Args:
        person_name (str): the name of the person saying hello

    Returns:
        dict: A dictionary containing the results of the query.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'query_result' key with an array of result rows.
              If 'error', includes an 'error_message' key.
    """
    return graphdb.send_query("RETURN 'Hello to you, ' + $person_name AS reply",
    {
        "person_name": person_name
    })


In [14]:
say_hello("Rajesh Thakur")

{'status': 'success',
 'query_result': [{'reply': 'Hello to you, Rajesh Thakur'}]}

In [15]:
say_hello("RETURN Rajesh Thakur")

{'status': 'success',
 'query_result': [{'reply': 'Hello to you, RETURN Rajesh Thakur'}]}

In [16]:
# Define the Cypher Agent
hello_agent = Agent(
    name="hello_agent_v1",
    model=llm_small, # defined earlier in a variable
    description="Has friendly chats with a user.",
    instruction="""You are a helpful assistant, chatting with a user. 
                Be polite and friendly, introducing yourself and asking who the user is. 

                If the user provides their name, use the 'say_hello' tool to get a custom greeting.
                If the tool returns an error, inform the user politely. 
                If the tool is successful, present the reply.
                """,
    tools=[say_hello], # Pass the function directly
)

print(f"Agent '{hello_agent.name}' created.")

Agent 'hello_agent_v1' created.


In [17]:
# Define the Cypher Agent
hello_agent1 = Agent(
    name="hello_agent_v2",
    model=llm_small, # defined earlier in a variable
    description="Has friendly chats with a user.",
    instruction="""You are a helpful assistant, chatting with a user. 
                Be polite and friendly, introducing yourself and asking who the user is. 

                If the user provides their name, use the 'say_hello' tool to get a custom greeting.
                If the tool returns an error, inform the user politely. 
                If the tool is successful, present the reply.
                """,
    tools=[say_hello], # Pass the function directly
)

print(f"Agent '{hello_agent.name}' created.")

Agent 'hello_agent_v1' created.


# Run The Agent

To run an agent, you'll need some additional components namely an execution environment and memory.


### Create the Runner and SessionService


Let's assume we have a single user talking to the agent in a single session. Let's create this user, the session and the runner:
* `SessionService`: Responsible for managing conversation history and state for different users and sessions. The `InMemorySessionService` is a simple implementation that stores everything in memory, suitable for testing and simple applications. It keeps track of the messages exchanged.  
* `Runner`: The engine that orchestrates the interaction flow. It takes user input, routes it to the appropriate agent, manages calls to the LLM and tools based on the agent's logic, handles session updates via the `SessionService`, and yields events representing the progress of the interaction.

In [18]:
app_name = hello_agent.name + "_app"
user_id = hello_agent.name + "_user_1"
session_id = hello_agent.name + "_session_1"


# Initialize a session service and a session
session_service = InMemorySessionService()
await session_service.create_session(app_name = app_name,
                                    user_id = user_id,
                                    session_id = session_id)

# Create the runner
runner = Runner(
    agent = hello_agent,
    app_name = app_name,
    session_service = session_service
)



# Run the Agent


Here's what's happening:
 
1. Package the user query into the ADK `Content` format.
2. Call`runner.run_async` (providing it with user/session context and the new message)
4. Iterate through the **Events** yielded by the runner. Events represent steps in the agent's execution (e.g., tool call requested, tool result received, intermediate LLM thought, final response).  
5. Identify and print the **final response** event using `event.is_final_response()`.

**Why `async`?** Interactions with LLMs and potentially tools (like external APIs) are I/O-bound operations. Using `asyncio` allows the program to handle these operations efficiently without blocking execution.


In [19]:
user_message = "Hello, I'm Rajesh Thakur."

content = types.Content(role='user', parts=[types.Part(text=user_message)])

final_response_text = "Agent did not produce a final response." # Default will be replaced if the agent produces a final response.

verbose = True

async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
    if verbose:
        print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")
    if event.is_final_response():
        if event.content and event.content.parts:
            final_response_text = event.content.parts[0].text ## # Assuming text response in the first part

        elif event.actions and event.actions.escalate:
            final_response_text = f"Agent escalated: {event.error_message or 'No specific message provided.'}"
        break
print(f"<<< Agent Response: {final_response_text}")

  [Event] Author: hello_agent_v1, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'person_name': 'Rajesh Thakur'
    },
    id='call_HtVq31j7Z7UrwwZX6Aa6gpfi',
    name='say_hello'
  )
)] role='model'
  [Event] Author: hello_agent_v1, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='call_HtVq31j7Z7UrwwZX6Aa6gpfi',
    name='say_hello',
    response={
      'query_result': [
        {
          'reply': 'Hello to you, Rajesh Thakur'
        },
      ],
      'status': 'success'
    }
  )
)] role='user'
  [Event] Author: hello_agent_v1, Type: Event, Final: True, Content: parts=[Part(
  text="Hello to you, Rajesh Thakur! It's great to meet you. How can I assist you today?"
)] role='model'
<<< Agent Response: Hello to you, Rajesh Thakur! It's great to meet you. How can I assist you today?


# Create Helper Class: AgentCaller

In [ ]:
# agent_caller = AgentCaller(agent, runner, user_id, session_id)
# agent_caller1 = AgentCaller(agent1, runner1, user_id1, session_id1)

In [20]:
class AgentCaller:
    """A simple wrapper class for interacting with an ADK agent."""
    
    def __init__(self, agent: Agent, runner: Runner, 
                 user_id: str, session_id: str):
        """Initialize the AgentCaller with required components."""
        self.agent = agent
        self.runner = runner
        self.user_id = user_id
        self.session_id = session_id


    def get_session(self):
        return self.runner.session_service.get_session(app_name=self.runner.app_name, user_id=self.user_id, session_id=self.session_id)

    
    async def call(self, user_message: str, verbose: bool = False):
        """Call the agent with a query and return the response."""
        print(f"\n>>> User Message: {user_message}")

        # Prepare the user's message in ADK format
        content = types.Content(role='user', parts=[types.Part(text=user_message)])

        final_response_text = "Agent did not produce a final response." 
        
        # Key Concept: run_async executes the agent logic and yields Events.
        # We iterate through events to find the final answer.
        async for event in self.runner.run_async(user_id=self.user_id, session_id=self.session_id, new_message=content):
            # You can uncomment the line below to see *all* events during execution
            if verbose:
                print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

            # Key Concept: is_final_response() marks the concluding message for the turn.
            if event.is_final_response():
                if event.content and event.content.parts:
                    # Assuming text response in the first part
                    final_response_text = event.content.parts[0].text
                elif event.actions and event.actions.escalate: # Handle potential errors/escalations
                    final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
                break # Stop processing events once the final response is found

        print(f"<<< Agent Response: {final_response_text}")
        return final_response_text


In [21]:
async def make_agent_caller(agent: Agent, initial_state: Optional[Dict[str, Any]] = {}) -> AgentCaller:
    """Create and return an AgentCaller instance for the given agent."""
    app_name = agent.name + "_app"
    user_id = agent.name + "_user"
    session_id = agent.name + "_session_01"
    
    # Initialize a session service and a session
    session_service = InMemorySessionService()
    await session_service.create_session(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id,
        state=initial_state
    )
    
    runner = Runner(
        agent=agent,
        app_name=app_name,
        session_service=session_service
    )
    
    return AgentCaller(agent, runner, user_id, session_id)


In [23]:
hello_agent_caller = await make_agent_caller(hello_agent)
hello_agent_caller1 = await make_agent_caller(hello_agent1)
async def run_conversation():
    await hello_agent_caller.call("Hello I'm ABK", verbose=True)

    await hello_agent_caller.call("I am excited")

In [24]:
await run_conversation()


>>> User Message: Hello I'm ABK
  [Event] Author: hello_agent_v1, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'person_name': 'ABK'
    },
    id='call_wr0aZJNZ4Ymp084YfYvOwPpv',
    name='say_hello'
  )
)] role='model'
  [Event] Author: hello_agent_v1, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='call_wr0aZJNZ4Ymp084YfYvOwPpv',
    name='say_hello',
    response={
      'query_result': [
        {
          'reply': 'Hello to you, ABK'
        },
      ],
      'status': 'success'
    }
  )
)] role='user'
  [Event] Author: hello_agent_v1, Type: Event, Final: True, Content: parts=[Part(
  text="Hello to you, ABK! It's great to meet you. How can I assist you today?"
)] role='model'
<<< Agent Response: Hello to you, ABK! It's great to meet you. How can I assist you today?

>>> User Message: I am excited
<<< Agent Response: That's wonderful to hear, ABK! Excitement is always a great feeli

{"id":"ff8081819782e69e0198fe9c0e627a1d","name":"Apple MacBook Pro 16","createdAt":"2025-08-31T05:31:36.930+00:00","data":{"year":2019,"price":1849.99,"CPU model":"Intel Core i9","Hard disk size":"1 TB"}}
